In [1]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)

import mneflow as mf
import mneflow
import tensorflow as tf
import numpy as np
from mneflow.models import BaseModel, LFCNN
from utils.machine_learning.designer import ModelDesign, ParallelDesign, LayerDesign
from utils.machine_learning.analyzer import ModelAnalyzer
from mneflow.layers import DeMixing, LFTConv, TempPooling, Dense, VARConv
from mne.datasets import multimodal
import mne
from utils.machine_learning.confusion import ConfusionEstimator
from utils.machine_learning import one_hot_decoder
import sklearn.metrics as sm
import tensorflow.keras.regularizers as k_reg
from tensorflow.keras.layers import SeparableConv2D, Conv2D, DepthwiseConv2D
from tensorflow.keras.layers import Flatten, Dropout, BatchNormalization
from tensorflow.keras import constraints, layers
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Activation, Permute, Dropout
from mne import channels, evoked, create_info

from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D

from scipy.signal import freqz, welch
from scipy.stats import spearmanr

2022-03-12 22:46:50.700835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 22:46:50.700867: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [131]:
n_times = 200
n_channels = 204
out_dim = 4

specs = dict()
specs.setdefault('filter_length', 7)
specs.setdefault('n_latent', 32)
specs.setdefault('pooling', 3)
specs.setdefault('stride', 3)
specs.setdefault('padding', 'SAME')
specs.setdefault('pool_type', 'max')
specs.setdefault('nonlin', tf.nn.relu)
specs.setdefault('l1', 3e-4)
specs.setdefault('l2', 0)
specs.setdefault('l1_scope', ['fc', 'demix', 'lf_conv'])
specs.setdefault('l2_scope', [])
specs.setdefault('maxnorm_scope', [])
specs.setdefault('dropout', .4)

lfcnnd = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    DeMixing(size=specs['n_latent'], nonlin=tf.identity, axis=3, specs=specs),
    LFTConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    TempPooling(
        pooling=specs['pooling'],
        pool_type=specs['pool_type'],
        stride=specs['stride'],
        padding=specs['padding'],
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)


varcnnd = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    DeMixing(size=specs['n_latent'], nonlin=tf.identity, axis=3, specs=specs),
    VARConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    TempPooling(
        pooling=specs['pooling'],
        pool_type=specs['pool_type'],
        stride=specs['stride'],
        padding=specs['padding'],
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)

Deep4_d = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    LayerDesign(tf.transpose, [0,3,2,1]),
    DepthwiseConv2D(
        kernel_size=(1, specs['filter_length']),
        depth_multiplier = specs['n_latent'],
        strides=1,
        padding=specs['padding'],
        activation = tf.identity,
        kernel_initializer="he_uniform",
        bias_initializer=Constant(0.1),
        data_format="channels_last",
        kernel_regularizer=k_reg.l2(specs['l2'])
        #kernel_constraint="maxnorm"
    ),
    *[ModelDesign(
        Conv2D(
            filters=specs['n_latent'],
            kernel_size=(n_channels, 1),
            strides=1,
            padding=specs['padding'],
            activation=specs['nonlin'],
            kernel_initializer="he_uniform",
            bias_initializer=Constant(0.1),
            data_format="channels_last",
            #data_format="channels_first",
            kernel_regularizer=k_reg.l2(specs['l2'])
        ),
        TempPooling(
            pooling=specs['pooling'],
            pool_type="avg",
            stride=specs['stride'],
            padding='SAME',
        )
    ) for _ in range(4)],
    Dense(size=out_dim, nonlin=tf.softmax)
)


specs['pooling'] = 75
specs['stride'] = 15
FBCSP_ShallowNet_d = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    LayerDesign(tf.transpose, [0,3,2,1]),
    DepthwiseConv2D(
        kernel_size=(1, specs['filter_length']),
        depth_multiplier = specs['n_latent'],
        strides=1,
        padding="VALID",
        activation = tf.identity,
        kernel_initializer="he_uniform",
        bias_initializer=Constant(0.1),
        data_format="channels_last",
        kernel_regularizer=k_reg.l2(specs['l2'])
        #kernel_constraint="maxnorm"
    ),
    Conv2D(
        filters=specs['n_latent'],
        kernel_size=(n_channels, 1),
        strides=1,
        padding="VALID",
        activation = tf.square,
        kernel_initializer="he_uniform",
        bias_initializer=Constant(0.1),
        data_format="channels_last",
        #data_format="channels_first",
        kernel_regularizer=k_reg.l2(specs['l2'])
    ),
    TempPooling(
        pooling=specs['pooling'],
        pool_type="avg",
        stride=specs['stride'],
        padding='SAME',
    )
)

EEGNet_d = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    LayerDesign(tf.transpose, [0,3,2,1]),
    Conv2D(
        specs['n_latent'], 
        (2, specs['filter_length']), 
        padding = specs['padding'],
        use_bias = False
    ),
    BatchNormalization(axis = 1),
    DepthwiseConv2D(
        (n_channels, 1), 
        use_bias = False, 
        depth_multiplier = 1,
        depthwise_constraint = constraints.MaxNorm(1.)
    ),
    BatchNormalization(),
    Activation('elu'),
    AveragePooling2D((1, specs['pooling'])),
    Dropout(specs['dropout']),
    SeparableConv2D(
        specs['n_latent'], 
        (1, specs['filter_length']//specs["pooling"]),
        use_bias = False,
        padding = 'same'
    ),
    BatchNormalization(),
    Activation('elu'),
    AveragePooling2D((1, specs['pooling']*2)),
    Dropout(specs['dropout']),
    Flatten(),
    Dense(size=out_dim)
)

EEGNet_d().shape

Setting reg for fc, to l1
Setting reg for fc, to l1


TensorShape([None, 4])

In [2]:
mne.set_log_level(verbose='CRITICAL')
fname_raw = os.path.join(multimodal.data_path(), 'multimodal_raw.fif')
raw = mne.io.read_raw_fif(fname_raw)

cond = raw.acqparser.get_condition(raw, None)
# get the list of condition names
condition_names = [k for c in cond for k,v in c['event_id'].items()]
epochs_list = [mne.Epochs(raw, **c) for c in cond]
epochs = mne.concatenate_epochs(epochs_list)
print(epochs.info)
epochs = epochs.pick_types(meg='grad')

X = np.array([])
Y = list()
for i, epochs in enumerate(epochs_list):
    data = epochs.get_data()
    if i == 0:
        X = data.copy()
    else:
        X = np.append(X, data, axis=0)
    Y += [i for _ in range(data.shape[0])]

Y = np.array(Y)

sample_matrix = X[0].copy()

X = np.array([X[i, epochs._channel_type_idx['grad'], :] for i, _ in enumerate(X)])


<Info | 22 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 acq_stim: 5 10.000000 500.000000 6 10.000000 500.000000
 bads: []
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 GRAD, 102 MAG, 9 STIM, 1 EOG
 custom_ref_applied: False
 description: Vectorview system at BioMag
 dev_head_t: MEG device -> head transform
 dig: 21 items (3 Cardinal, 4 HPI, 14 Extra)
 events: 1 item (list)
 experimenter: neuromag
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2011-04-26 11:33:18 UTC
 meas_id: 4 items (dict)
 nchan: 316
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: grad_ssp_upright.fif : PCA-v1: on, grad_ssp_upright.fif : ...
 sfreq: 600.6 Hz
 subject_info: 9 items (dict)
>


In [3]:
import_opt = dict(savepath='../tfr/',
                out_name='mne_sample_epochs',
                fs=600,
                input_type='trials',
                target_type='int',
                picks={'meg':'grad'},
                scale=True,  # apply baseline_scaling
                crop_baseline=True,  # remove baseline interval after scaling
                decimate=None,
                scale_interval=(0, 60),  # indices in time axis corresponding to baseline interval
            #   n_folds=5,  # validation set size set to 20% of all data
                n_folds=5,
                overwrite=True,
                segment=False,
                test_set='holdout'
)

lf_params = dict(n_latent=32, #number of latent factors
                filter_length=17, #convolutional filter length in time samples
                nonlin = tf.nn.relu,
                padding = 'SAME',
                pooling = 5,#pooling factor
                stride = 5, #stride parameter for pooling layer
                pool_type='max',
                model_path = import_opt['savepath'],
                dropout = .5,
                l1_scope = ["weights"],
                l1=3e-3
)


meta = mf.produce_tfrecords((X, Y), **import_opt)  
dataset = mf.Dataset(meta, train_batch=100)

importing from tuple
input shapes: X- (940, 204, 361) targets- (940, 1)
Preprocessing:
Scaling to interval 0.0 - 60.0
Splitting sets
Preprocessed: (940, 1, 361, 204) (940, 8) folds: 6 x 156
936
Prepocessed sample shape: (1, 361, 204)
Target shape actual/metadata:  (8,) (8,)
Saving TFRecord# 0


2022-03-12 22:47:29.777468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-12 22:47:29.777497: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-12 22:47:29.777517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arcolinux-machine): /proc/driver/nvidia/version does not exist
2022-03-12 22:47:29.777774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
class LFCNNm(LFCNN):
    def __init__(self, Dataset, specs=dict()):
        self.scope = 'lfcnn'
        specs.setdefault('filter_length', 7)
        specs.setdefault('n_latent', 32)
        specs.setdefault('pooling', 2)
        specs.setdefault('stride', 2)
        specs.setdefault('padding', 'SAME')
        specs.setdefault('pool_type', 'max')
        specs.setdefault('nonlin', tf.nn.relu)
        specs.setdefault('l1', 3e-4)
        specs.setdefault('l2', 0)
        specs.setdefault('l1_scope', ['fc', 'demix', 'lf_conv'])
        specs.setdefault('l2_scope', [])
        specs.setdefault('maxnorm_scope', [])
        
        super(LFCNNm, self).__init__(Dataset, specs)

    def build_graph(self):
        
        # self.dmx
        # self.tconv
        # self.pool
        # self.fin_fc
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     DeMixing(size=self.specs['n_latent'], nonlin=tf.identity, axis=3, specs=self.specs),
        #     LFTConv(
        #         size=self.specs['n_latent'],
        #         nonlin=self.specs['nonlin'],
        #         filter_length=self.specs['filter_length'],
        #         padding=self.specs['padding'],
        #         specs=self.specs
        #     ),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     tf.keras.layers.DepthwiseConv2D((1, 361), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        
        self.design = ModelDesign(
            self.inputs,
            DeMixing(size=self.specs['n_latent'], nonlin=tf.identity, axis=3, specs=self.specs),
            LFTConv(
                size=self.specs['n_latent'],
                nonlin=self.specs['nonlin'],
                filter_length=self.specs['filter_length'],
                padding=self.specs['padding'],
                specs=self.specs
            ),
            tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
            LayerDesign(tf.transpose, [0, 1, 3, 2]),
            tf.keras.layers.DepthwiseConv2D((1, 32), padding='valid', activation='relu', kernel_regularizer='l1'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1')
        )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     tf.keras.layers.Conv2D(self.specs['n_latent'], (1, 204), padding='same', kernel_regularizer='l1'),
        #     tf.keras.layers.DepthwiseConv2D((1, self.specs['filter_length']), padding='same', kernel_regularizer='l1'),
        #     # LayerDesign(tf.nn.avg_pool2d, ksize=(1, 1, 10, 1), strides=(1, 1, 10, 1), padding='SAME'),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     # tf.keras.layers.Flatten(),
        #     # tf.keras.layers.Dense(self.out_dim),
        #     tf.keras.layers.DepthwiseConv2D((1, 361), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     LayerDesign(tf.squeeze, axis=1),
        #     # tf.keras.layers.Bidirectional(
        #     tf.keras.layers.LSTM(
        #         self.specs['n_latent'],
        #         bias_regularizer='l1',
        #         return_sequences=True,
        #         kernel_regularizer=tf.keras.regularizers.L1(.01),
        #         recurrent_regularizer=tf.keras.regularizers.L1(.01),
        #         dropout=0.4,
        #         recurrent_dropout=0.4,
        #     ),
        #     #     merge_mode='sum'
        #     # ),
        #     LayerDesign(tf.expand_dims, axis=1),
        #     LFTConv(
        #         size=self.specs['n_latent'],
        #         nonlin=self.specs['nonlin'],
        #         filter_length=self.specs['filter_length'],
        #         padding=self.specs['padding'],
        #         specs=self.specs
        #     ),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     LayerDesign(tf.transpose, [0, 1, 3, 2]),
        #     tf.keras.layers.DepthwiseConv2D((1, 32), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     LayerDesign(tf.squeeze, axis=1),
        #     tf.keras.layers.Bidirectional(
        #         tf.keras.layers.LSTM(
        #             self.specs['n_latent'],
        #             bias_regularizer='l1',
        #             return_sequences=True,
        #             kernel_regularizer=tf.keras.regularizers.L1(.01),
        #             recurrent_regularizer=tf.keras.regularizers.L1(.01),
        #             dropout=0.4,
        #             recurrent_dropout=0.4,
        #         ),
        #         merge_mode='sum'
        #     ),
        #     LayerDesign(tf.expand_dims, axis=1),
        #     tf.keras.layers.DepthwiseConv2D((1, self.specs['filter_length']), padding='same', kernel_regularizer='l1'),
        #     # LayerDesign(tf.nn.avg_pool2d, ksize=(1, 1, 10, 1), strides=(1, 1, 10, 1), padding='SAME'),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     # tf.keras.layers.Flatten(),
        #     # tf.keras.layers.Dense(self.out_dim),
        #     tf.keras.layers.DepthwiseConv2D((1, 361), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     DeMixing(size=self.specs['n_latent'], nonlin=tf.identity, axis=3, specs=self.specs),
        #     ParallelDesign(
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']//2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length'],
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']*2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #     ),
        #     TempPooling(
        #         pooling=self.specs['pooling'],
        #         pool_type=self.specs['pool_type'],
        #         stride=self.specs['stride'],
        #         padding=self.specs['padding'],
        #     ),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     Dense(size=self.out_dim, nonlin=tf.identity, specs=self.specs)
        # )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     LayerDesign(tf.squeeze, axis=1),
        #     tf.keras.layers.Bidirectional(
        #         tf.keras.layers.LSTM(
        #             self.specs['n_latent'],
        #             bias_regularizer='l1',
        #             return_sequences=True,
        #             kernel_regularizer=tf.keras.regularizers.L1(.01),
        #             recurrent_regularizer=tf.keras.regularizers.L1(.01),
        #             dropout=0.4,
        #             recurrent_dropout=0.4,
        #         ),
        #         merge_mode='sum'
        #     ),
        #     LayerDesign(tf.expand_dims, axis=1),
        #     ParallelDesign(
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']//2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length'],
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']*2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #     ),
            
        #     TempPooling(
        #         pooling=self.specs['pooling'],
        #         pool_type=self.specs['pool_type'],
        #         stride=self.specs['stride'],
        #         padding=self.specs['padding'],
        #     ),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     Dense(size=self.out_dim, nonlin=tf.identity, specs=self.specs)
        # )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     LayerDesign(tf.squeeze, axis=1),
        #     tf.keras.layers.Bidirectional(
        #         tf.keras.layers.LSTM(
        #             self.specs['n_latent'],
        #             bias_regularizer='l1',
        #             return_sequences=True,
        #             kernel_regularizer=tf.keras.regularizers.L1(.01),
        #             recurrent_regularizer=tf.keras.regularizers.L1(.01),
        #             dropout=0.4,
        #             recurrent_dropout=0.4,
        #         ),
        #         merge_mode='sum'
        #     ),
        #     LayerDesign(tf.expand_dims, axis=1),
        #     ParallelDesign(
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']//2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length'],
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #         LFTConv(
        #             size=self.specs['n_latent'],
        #             nonlin=self.specs['nonlin'],
        #             filter_length=self.specs['filter_length']*2,
        #             padding=self.specs['padding'],
        #             specs=self.specs
        #         ),
        #     ),
        #     tf.keras.layers.DepthwiseConv2D((1, 361), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        
        # self.design = ModelDesign(
        #     self.inputs,
        #     LayerDesign(tf.squeeze, axis=1),
        #     tf.keras.layers.Bidirectional(
        #         tf.keras.layers.LSTM(
        #             self.specs['n_latent'],
        #             bias_regularizer='l1',
        #             return_sequences=True,
        #             kernel_regularizer=tf.keras.regularizers.L1(.01),
        #             recurrent_regularizer=tf.keras.regularizers.L1(.01),
        #             dropout=0.4,
        #             recurrent_dropout=0.4,
        #         ),
        #         merge_mode='sum'
        #     ),
        #     LayerDesign(tf.expand_dims, axis=1),
        #     LFTConv(
        #         size=self.specs['n_latent'],
        #         nonlin=self.specs['nonlin'],
        #         filter_length=self.specs['filter_length']*2,
        #         padding=self.specs['padding'],
        #         specs=self.specs
        #     ),
        #     tf.keras.layers.DepthwiseConv2D((1, 361), padding='valid', activation='relu', kernel_regularizer='l1'),
        #     tf.keras.layers.Dropout(self.specs['dropout'], noise_shape=None),
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(self.out_dim, kernel_regularizer='l1'),
        # )
        

        return self.design()


In [6]:
model = LFCNNm(dataset, lf_params)
model.build_graph().shape

Setting reg for tconv, to l1
Built: tconv input: (None, 1, 361, 32)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 361, 32)


TensorShape([None, 8])

In [15]:
model = LFCNNm(dataset, lf_params)
model.build()
model.train(n_epochs=100, eval_step=100, early_stopping=10)

Setting reg for dmx, to l1
Built: dmx input: (None, 1, 361, 204)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 361, 32)
Input shape: (1, 361, 204)
y_pred: (None, 8)
Initialization complete!
Epoch 1/100
100/100 - 4s - loss: 5.0309 - cat_ACC: 0.1231 - val_loss: 4.4111 - val_cat_ACC: 0.1278 - 4s/epoch - 43ms/step
Epoch 2/100
100/100 - 3s - loss: 3.9116 - cat_ACC: 0.1177 - val_loss: 3.4551 - val_cat_ACC: 0.1278 - 3s/epoch - 33ms/step
Epoch 3/100
100/100 - 3s - loss: 3.1135 - cat_ACC: 0.1177 - val_loss: 2.8019 - val_cat_ACC: 0.1278 - 3s/epoch - 35ms/step
Epoch 4/100
100/100 - 3s - loss: 2.5990 - cat_ACC: 0.1099 - val_loss: 2.3926 - val_cat_ACC: 0.1278 - 3s/epoch - 35ms/step
Epoch 5/100
100/100 - 3s - loss: 2.2871 - cat_ACC: 0.1187 - val_loss: 2.1764 - val_cat_ACC: 0.1278 - 3s/epoch - 35ms/step
Epoch 6/100
100/100 - 3s - loss: 2.1287 - cat_ACC: 0.1316 - val_loss: 2.0674 - val_cat_ACC: 0.1805 - 3s/epoch - 34ms/step
Epoch 7/100
100/100 - 4s - loss: 2.0591 - cat_ACC: 0.1739 - val_l

KeyboardInterrupt: 

In [148]:
model = LFCNN(dataset, lf_params)
model.build()
model.train(n_epochs=100, eval_step=100, early_stopping=5)

Setting reg for dmx, to l1
Built: dmx input: (None, 1, 361, 204)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 361, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 73, 32)
Input shape: (1, 361, 204)
y_pred: (None, 8)
Initialization complete!
Epoch 1/100
100/100 - 4s - loss: 5.3149 - cat_ACC: 0.1720 - val_loss: 4.9199 - val_cat_ACC: 0.1825 - 4s/epoch - 44ms/step
Epoch 2/100
100/100 - 3s - loss: 4.5731 - cat_ACC: 0.3550 - val_loss: 4.3783 - val_cat_ACC: 0.3333 - 3s/epoch - 32ms/step
Epoch 3/100
100/100 - 3s - loss: 3.9398 - cat_ACC: 0.5304 - val_loss: 3.8819 - val_cat_ACC: 0.5238 - 3s/epoch - 30ms/step
Epoch 4/100
100/100 - 3s - loss: 3.4060 - cat_ACC: 0.7010 - val_loss: 3.4279 - val_cat_ACC: 0.7063 - 3s/epoch - 32ms/step
Epoch 5/100
100/100 - 3s - loss: 2.9393 - cat_ACC: 0.8137 - val_loss: 3.0576 - val_cat_ACC: 0.7063 - 3s/epoch - 32ms/step
Epoch 6/100
100/100 - 3s - loss: 2.5730 - cat_ACC: 0.8838 - val_loss: 2.7648 - val_cat_ACC: 0.7460 - 3s/epoch - 29ms/step
Epoc